In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import random
import matplotlib.pyplot as plt

seed_value=3
random.seed(seed_value)

distribution="normal Printed Circuit Board" # 正規分布（多層回路基盤を作る）

#要素
#注文
odder_mean = 10  # オーダー数平均値
odder_std_dev = 2  # オーダー数標準偏差
odder_size = 1000  # オーダー数样本サイズ

lot_mean = 3  # 注文された製品のロット数平均値
lot_std_dev = 1  # 注文された製品のロット数標準偏差
lot_size = 1000  # 注文された製品のロット数样本サイズ

#機械加工工程設定
m_low=1 #最小機械番号
m_high=14 #最大機械番号(m_high-1)

M1_t_unit = 0.5  # M1の1単位操作時間（銅箔付きの絶縁板1を取る）
M2_t_unit = 0.5  # M2の1単位操作時間（銅箔付きの絶縁板2を取る）
M3_t_unit = 2  # M3の1単位操作時間（表面の汚れや油分を除去し）
M4_t_unit = 8  # M4の1単位操作時間（パターン転写、フォトリソグラフィ方式）
M5_t_unit = 7  # M5の1単位操作時間（パターン転写、ダイレクト印刷方式）
M6_t_base = 250  # M6のベース操作時間（エッチングとレジスト除去）
M7_t_base = 600  # M7のベース操作時間（積層）
M8_t_unit = 3 # M8の1単位操作時間（ドリル加工）
M9_t_base = 750  # M9のベース操作時間（スルーホールめっき）
M10_t_unit = 5 # M10の1単位操作時間（ソルダーレジスト塗布）
M11_t_unit = 4 # M11の1単位操作時間（表面処理）
M12_t_unit = 2 # M12の1単位操作時間（電気検査）
M13_t_unit = 1 # M13の1単位操作時間（包装）

#加工機械順番
MS_J=np.array([[1,3,4,6,7,8,9,10,11,12,13,0,0,0,0,0,0],#1層
               [2,3,5,6,7,8,9,10,11,12,13,0,0,0,0,0,0],#1層
               [1,3,4,6,5,6,7,8,9,10,11,12,13,0,0,0,0],#2層
               [2,3,5,6,4,6,7,8,9,10,11,12,13,0,0,0,0],#2層
               [1,3,5,6,4,6,7,8,9,4,6,10,11,12,13,0,0],#3層
               [2,3,5,6,4,6,7,8,9,4,6,5,6,10,11,12,13]])#4層

p_number_max=17 #すべての種類の製品に応じて最大プロセス数
count_type=6 #製品の種類数

#print(MS_J)

gene=10000

for g in np.arange(0,gene):
    
    odder_data = np.round(np.random.normal(odder_mean, odder_std_dev, odder_size))
    odder_data = odder_data[odder_data > 0]  # マイナスやゼロを除外
    odder_number = int(random.choice(odder_data))  # 実際のオーダー数を1つランダムに選ぶ
    #print("ジョブ数は：",odder_number)
    #plt.hist(odder_data, bins=30, density=True, alpha=0.6, color='blue', edgecolor='black')
    
    #ロットデータ
    lot_data = np.round(np.random.normal(lot_mean, lot_std_dev, lot_size))
    lot_data = lot_data[lot_data > 0]  # ゼロやマイナスを除外
    
    odder = np.zeros((odder_number, count_type))#オーダー原始データ生成

    #各オーダーの製品種類と製品ロット数固定
    for i in range(odder_number):
        for j in range(count_type):
            if random.random() >= 0.3:
                odder[i, j] = random.choice(lot_data)
                
    #print(odder)
    
    product_count = np.sum(odder, axis=0) * 10
    #print(product_count)

    #各プロセス要する操作時間固定
    T_J=np.zeros((int(count_type),int(p_number_max)))
    #print(T_J)

    for i in np.arange(0, count_type):
        for j in np.arange(0, p_number_max):
            if MS_J[i, j] == 1:
                T_J[i, j] = M1_t_unit * product_count[i]
            elif MS_J[i, j] == 2:
                T_J[i, j] = M2_t_unit * product_count[i]
            elif MS_J[i, j] == 3:
                T_J[i, j] = M3_t_unit * product_count[i]
            elif MS_J[i, j] == 4:
                T_J[i, j] = M4_t_unit * product_count[i]
            elif MS_J[i, j] == 5:
                T_J[i, j] = M5_t_unit * product_count[i]
            elif MS_J[i, j] == 6:
                T_J[i, j] = M6_t_base
            elif MS_J[i, j] == 7:
                T_J[i, j] = M7_t_base
            elif MS_J[i, j] == 8:
                T_J[i, j] = M8_t_unit * product_count[i]
            elif MS_J[i, j] == 9:
                T_J[i, j] = M9_t_base
            elif MS_J[i, j] == 10:
                T_J[i, j] = M10_t_unit * product_count[i]
            elif MS_J[i, j] == 11:
                T_J[i, j] = M11_t_unit * product_count[i]
            elif MS_J[i, j] == 12:
                T_J[i, j] = M12_t_unit * product_count[i]
            elif MS_J[i, j] == 13:
                T_J[i, j] = M13_t_unit * product_count[i]       
    #print('ジョブセットの操作時間:\n',T_J)

    JOBSET_0=np.zeros((int(count_type),int(p_number_max)))
    JOBSET=pd.DataFrame(JOBSET_0, columns=[f"process{i+1}" for i in range(int(p_number_max))], index=[f"job{i+1}" for i in range(int(count_type))])
    JOBSET=JOBSET.astype(str)

    for i in np.arange(0,count_type):
        for j in np.arange(0,p_number_max):
            m=MS_J[i,j]
            t=T_J[i,j]
            JOBSET.iloc[i,j]=str(m),str(t)    
    #print('自動生成したジョブセット：\n',JOBSET)
    
    str1="D://pysrc/wang_data/jobset/"+str(distribution)+"/"
    odder_element="odder_mean["+str(odder_mean)+"],odder_std_dev["+str(odder_std_dev)+"]/"
    lot_element="lot_mean["+str(lot_mean)+"],lot_std_dev["+str(lot_std_dev)+"]/"
    m_element="machine["+str(m_high-1)+"]/"
    file="seed["+str(seed_value)+"]/"
    name="["+str(count_type)+']r['+str(p_number_max)+']c,'+str(g)+'gene'
    str2=".csv"

    filename=str1+odder_element+lot_element+m_element+file+name+str2
    
    JOBSET.to_csv(filename,index=True)
